In [1]:
from loading_data import queryDB
import pandas as pd

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
annotations = queryDB('select * from annotations limit 100')

In [3]:
def compute_overlap(annotationA, annotationB):
    # if there is no overlap in x dimension
    if annotationB.x2 - annotationA.x1 < 0 or annotationA.x2 - annotationB.x1 < 0:
        return 0
    # if there is no overlap in y dimension
    if annotationB.y2 - annotationA.y1 < 0 or annotationA.y2 - annotationB.y1 < 0:
        return 0
    
    areaA = (annotationA.x2-annotationA.x1) * (annotationA.y2-annotationA.y1)
    areaB = (annotationB.x2-annotationB.x1) * (annotationB.y2-annotationB.y1)

    width = min(annotationA.x2,annotationB.x2) - min(annotationA.x1,annotationB.x1)
    height = min(annotationA.y2,annotationB.y2) - min(annotationA.y1,annotationB.y1)
    
    area_intersect = height * width
    iou = area_intersect / (areaA + areaB - area_intersect)
    
    return iou

In [4]:
a = annotations.iloc[0]
a.x1 = -1
a.x2 = 0
if compute_overlap(a,annotations.iloc[0]) != 0:
    print("ERROR")
    
a = annotations.iloc[0]
a.x1 = 0
a.x2 = 1000
a.y1 = 0
a.y2 = 1000
b = annotations.iloc[0]
b.x1 = 0
b.x2 = 1000
b.y1 = 0
b.y2 = 500

if compute_overlap(a,b) != .5:
    print("ERROR")

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [5]:
def score_predictions(validation,predictions,iou_thresh, concepts):
    # group predictions by video frames
    predictions = predictions.groupby(['videoid','timeinvideo'], sort=False)
    predictions = [df for _, df in predictions]
    
    # mapping annotations from prediction list to video/time
    frame_data = {}
    for i, group in enumerate(predictions):
        time = group.iloc[0]['timeinvideo']
        video = group.iloc[0]['videoid']
        frame_data[(video,time)] = i
    
    # group validation annotations by video frames
    validation = validation.groupby(['videoid','timeinvideo'], sort=False)
    validation = [df for _, df in validation]
    
    # initialise counters for each concept
    true_positives = dict(zip(concepts,[0] * len(concepts)))
    false_positives = dict(zip(concepts,[0] * len(concepts)))
    false_negatives = dict(zip(concepts,[0] * len(concepts)))
    
    # get true and false positives for each frame of validation data
    for group in validation:
        #get time and video for this set
        time = group.iloc[0]['timeinvideo']
        video = group.iloc[0]['videoid']
        
        #get corresponding predictions for this set
        predicted = predictions[frame_data[(video,time)]]
        
        #go through predictions and identify false positives/false negatives/ true_positives
        detected_predictions = []
        detected_truths = dict(zip(concepts,[0] * len(concepts)))
        for index, truth in group.iterrows():
            for index, prediction in predicted.iterrows():
                if prediction in detected_predictions or compute_overlap(truth,prediction) < iou_thresh:
                    continue
                detected_predictions.append(prediction)
                if truth.conceptid == prediction.conceptid:
                    true_positives[prediction.conceptid] += 1
                    detected_truths[prediction.conceptid] += 1
                else:
                    false_positives[prediction.conceptid] += 1
                    false_negatives[truth.conceptid] += 1
        for id in concepts:
            false_negatives[id] += len(group.loc[group['conceptid'] == id]) - detected_truths[id]
        #count number of annotations that were correct, wrong, and missed
        
    f1 = dict(zip(concepts,[0] * len(concepts)))
    for id in concepts:
        recall = true_positives[id] / (true_positives[id] + false_negatives[id])
        precision = true_positives[id] / (true_positives[id] + false_positives[id])

        f1[id] = 2*recall*precision / (precision+recall)
    
    return f1

In [6]:
match_annotations(annotations,annotations,.1,annotations.conceptid.unique())

{79: 1.0, 236: 1.0, 1009: 1.0, 2136: 1.0}

In [8]:
#DOESNT WORK IF VAL IS MISSING

In [9]:
#is it false positive if something is misidentified, or false positive as well as false negative?
# currently both!

In [10]:
queryDB('select * from annotations limit 1')

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,2178619,19,17,2136,773.44,614.0,474.0,690.0,512.0,1280.0,720.0,2019-01-27,19_773.44_ai.png,6945_773.44_box_ai.png,None,False,6945,23180


In [11]:
queryDB('select * from annotations where userid != 17 limit 1')

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,11107,29,11,1083,96.110808,82.0,285.0,145.984383,321.0,1600.0,900.0,2018-11-13,11107.png,11107_box.png,,False,11107,None


In [14]:
annotations.head()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,2178619,19,17,2136,773.44,614.0,474.0,690.0,512.0,1280.0,720.0,2019-01-27,19_773.44_ai.png,6945_773.44_box_ai.png,None,False,6945,23180
1,2178626,19,17,2136,773.48,614.0,478.0,690.0,516.0,1280.0,720.0,2019-01-27,19_773.48_ai.png,6945_773.48_box_ai.png,None,False,6945,23181
2,2178641,19,17,2136,773.51,615.0,481.0,691.0,519.0,1280.0,720.0,2019-01-27,19_773.51_ai.png,6945_773.51_box_ai.png,None,False,6945,23182
3,2178664,19,17,2136,773.54,615.0,484.0,691.0,522.0,1280.0,720.0,2019-01-27,19_773.54_ai.png,6945_773.54_box_ai.png,None,False,6945,23183
4,2178694,19,17,2136,773.58,616.0,488.0,692.0,526.0,1280.0,720.0,2019-01-27,19_773.58_ai.png,6945_773.58_box_ai.png,None,False,6945,23184
